In [31]:
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import openai
import random
from retrying import retry
import pyarrow as pa
import pyarrow.parquet as pq
from transformers import AutoTokenizer
import pandas as pd
import re

openai.api_base = "https://api.deepseek.com"


class OpenAIGPT:
    def __init__(self, model_name="deepseek-chat", keys_path=None):
        self.model_name = model_name
        with open(keys_path, encoding="utf-8", mode="r") as fr:
            self.keys = [line.strip() for line in fr if len(line.strip()) >= 4]

    def __post_process(self, response):
        return response["choices"][0]["message"]["content"]

    @retry(wait_fixed=300, stop_max_attempt_number=50)
    def __call__(self, message):
        if message is None or message == "":
            return False, "Your input is empty."

        # current_key = random.choice(self.keys)
        current_key = self.keys[0] if len(self.keys) == 1 else random.choice(self.keys)
        openai.api_key = current_key
        response = openai.ChatCompletion.create(
            model=self.model_name,
            messages=[{"role": "user", "content": message}],
            temperature=0.7,
            top_p=0.7,
            frequency_penalty=0.6,
            presence_penalty=0.6,
            n=1,
        )
        return self.__post_process(response)


igpt = OpenAIGPT(keys_path="../train/apikeys.txt")

In [32]:
# igpt('hello')

In [33]:
name = 's5'
data = pd.read_json(f'test_dataset/{name}.json')
data.head()

,prompt,content,expected,predicted,log_r
0,请根据以下新闻文本，预测三一重工股票的5天（短期）后对数收益率属于以下哪一类别：正面 / 负面,现在是2023-04-14 09:00:00+08:00\n前五个交易日上证指数的价格如下3...,负面,负面,-0.017105
1,请根据以下新闻文本，预测三一重工股票的5天（短期）后对数收益率属于以下哪一类别：正面 / 负面,现在是2023-04-25 09:00:00+08:00\n前五个交易日上证指数的价格如下3...,正面,负面,0.014185
2,请根据以下新闻文本，预测三一重工股票的5天（短期）后对数收益率属于以下哪一类别：正面 / 负面,现在是2023-04-25 09:00:00+08:00\n前五个交易日上证指数的价格如下3...,正面,负面,0.014185
3,请根据以下新闻文本，预测三一重工股票的5天（短期）后对数收益率属于以下哪一类别：正面 / 负面,现在是2023-04-26 09:00:00+08:00\n前五个交易日上证指数的价格如下3...,正面,负面,0.003083
4,请根据以下新闻文本，预测三一重工股票的5天（短期）后对数收益率属于以下哪一类别：正面 / 负面,现在是2023-04-26 09:00:00+08:00\n前五个交易日上证指数的价格如下3...,正面,负面,0.003083


In [34]:
data['label'], bins = pd.qcut(data['log_r'], 5, labels=False, retbins=True)

# Printing if there are any null values in 'log_r'
print(data['log_r'].isnull().any())

# Printing the rows where 'log_r' is null
print(data[data['log_r'].isnull()])

# Printing the bin ranges
print("Bin ranges:")
for i in range(len(bins) - 1):
    print(f"Bin {i}: {bins[i]} to {bins[i + 1]}")

False
Empty DataFrame
Columns: [prompt, content, expected, predicted, log_r, label]
Index: []
Bin ranges:
Bin 0: -0.198250128836639 to -0.036905895787131
Bin 1: -0.036905895787131 to -0.013489991983674001
Bin 2: -0.013489991983674001 to 0.004271936541049604
Bin 3: 0.004271936541049604 to 0.025463164780335
Bin 4: 0.025463164780335 to 0.174973102087353


In [35]:
def process_row(row):

    def match_choice(text):
        if not isinstance(text, str):
            return ''
        match = re.findall(r'.*?([A-G]+(?:[、, ]+[A-G]+)*)', text)
        if match:
            last_match = match[-1]
            return ''.join(re.split(r'[、, ]+', last_match))
        return ''

    def predict(prompt):
        igpt = OpenAIGPT(keys_path="../train/apikeys.txt")
        try:
            return (igpt(prompt))
        except openai.error.APIError as e:
            print(f"Error occurred with prompt: {prompt}")
            print(f"Error: {e}")
            # return ''

    idx, row_copy = row
    predicted = row_copy['predicted']
    prompt = ''
    if predicted == '正面':
        prompt = '这个股票的5天（短期）后对数收益率属于正面，请先结合新闻文本，分析这个答案是否正确（如果错误则选择 F. 负面），再进一步分析，给出你的投资建议，你必须作出以下选择： A. 中性 B. 正面 C. 极度正面, 在这几个选项中正面代表一定会投资，请谨慎判断'
    else:
        prompt = '这个股票的5天（短期）后对数收益率属于负面，请先结合新闻文本，分析这个答案是否正确（如果错误则选择 B. 正面），再进一步分析，给出你的投资建议，你必须作出以下选择： D. 中性 F. 负面 G. 极度负面'

    res = predict(f"{row_copy['content']}\n{prompt}")
    choice = match_choice(res)

    choice_mapping = {
        'A': '中性',
        'B': '正面',
        'C': '极度正面',
        'D': '中性',
        'F': '负面',
        'G': '极度负面'
    }

    choice_text = choice_mapping.get(choice, '')

    label_mapping = {0: "极度负面", 1: "负面", 2: "中性", 3: "正面", 4: "极度正面"}
    label = label_mapping[row_copy['label']]

    return {
        "content": row_copy['content'],
        "expected": label,
        "log_r": row_copy['log_r'],
        "response": res,
        "choice": choice,
        "predicted": choice_text
    }


def generate_dataset(data, max_samples=1400):
    results = []
    with ThreadPoolExecutor(max_workers=64) as executor:
        futures = [executor.submit(process_row, row) for row in data[:max_samples].iterrows()]
        for future in tqdm(futures, total=len(futures), desc="Processing rows"):
            result = future.result()
            if result:
                results.append(result)

    return results

In [36]:
process_data = generate_dataset(data)

Processing rows:  12%|█▏        | 155/1314 [01:12<03:59,  4.83it/s] 

Error occurred with prompt: 现在是2023-07-31 09:00:00+08:00
前五个交易日上证指数的价格如下3163.421,3157.469,3201.458,3228.087,3225.476

2023-07-04 07:44:04 新闻:【36.76亿元市值限售股今日解禁，大金重工、奥迪威、爱柯迪解禁市值居前】今日共有7家公司限售股解禁，合计解禁量为1.62亿股，按最新收盘价计算，合计解禁市值为36.76亿元。从解禁量来看，4家公司解禁股数超千万股。其中，大金重工、瑞茂通、奥迪威解禁量居前，解禁股数分别为0.82亿股、0.35亿股、0.25亿股。从解禁市值来看，4家公司解禁市值超亿元。其中，大金重工、奥迪威、爱柯迪解禁市值居前，解禁市值分别为27.84亿元、2.93亿元、2.92亿元。
2023-07-16 09:24:42 新闻:【下周合计解禁市值为920.34亿元，寒武纪-U解禁市值超320亿元】 Wind数据显示，除将上市的新股外，下周将有73只股票面临限售股解禁，合计解禁量为40.45亿股。按最新收盘价计算，合计解禁市值为920.34亿元，较本周解禁市值减少9.18%。在下周面临解禁的73只股票中，从解禁市值情况来看，Wind数据显示，按最新收盘价计算，寒武纪-U、中国国航、同庆楼解禁市值居前，分别为326.79亿元、89.69亿元、61.58亿元。（中证报）
2023-07-22 08:26:37 新闻:【下周将有71股解禁 合计解禁市值555.93亿元】下周将有71股解禁，按照最新收盘价计算，合计解禁市值555.93亿元。国博电子、科思股份、国光连锁、软通动力解禁市值居前。国博电子解禁市值规模最大，下周将有1.36亿股上市流通，主要为首发原股东限售股份和首发战略配售股份，解禁市值达99.51亿元。科思股份的解禁规模次之，下周将有1.03亿股上市流通，解禁股主要为首发原股东限售股份，解禁市值达79.85亿元。科润智控、华康医疗、建邦科技等9股的解禁压力较小，解禁市值均不足千万元。解禁比例来看，国光连锁、科思股份、科拓生物的解禁比例均超50%。其中国光连锁的解禁股数量占总股本比例最大，达87.82%。此外，泽璟制药-U、晶澳科技、建邦科技、华康医疗、领湃科技的解禁比例较小，均不到0.1%。
2023-07-25 0

Processing rows:  12%|█▏        | 158/1314 [01:16<05:16,  3.65it/s]

Error occurred with prompt: 现在是2023-08-01 09:00:00+08:00
前五个交易日上证指数的价格如下3157.469,3201.458,3228.087,3225.476,3206.741

2023-07-04 07:44:04 新闻:【36.76亿元市值限售股今日解禁，大金重工、奥迪威、爱柯迪解禁市值居前】今日共有7家公司限售股解禁，合计解禁量为1.62亿股，按最新收盘价计算，合计解禁市值为36.76亿元。从解禁量来看，4家公司解禁股数超千万股。其中，大金重工、瑞茂通、奥迪威解禁量居前，解禁股数分别为0.82亿股、0.35亿股、0.25亿股。从解禁市值来看，4家公司解禁市值超亿元。其中，大金重工、奥迪威、爱柯迪解禁市值居前，解禁市值分别为27.84亿元、2.93亿元、2.92亿元。
2023-07-16 09:24:42 新闻:【下周合计解禁市值为920.34亿元，寒武纪-U解禁市值超320亿元】 Wind数据显示，除将上市的新股外，下周将有73只股票面临限售股解禁，合计解禁量为40.45亿股。按最新收盘价计算，合计解禁市值为920.34亿元，较本周解禁市值减少9.18%。在下周面临解禁的73只股票中，从解禁市值情况来看，Wind数据显示，按最新收盘价计算，寒武纪-U、中国国航、同庆楼解禁市值居前，分别为326.79亿元、89.69亿元、61.58亿元。（中证报）
2023-07-22 08:26:37 新闻:【下周将有71股解禁 合计解禁市值555.93亿元】下周将有71股解禁，按照最新收盘价计算，合计解禁市值555.93亿元。国博电子、科思股份、国光连锁、软通动力解禁市值居前。国博电子解禁市值规模最大，下周将有1.36亿股上市流通，主要为首发原股东限售股份和首发战略配售股份，解禁市值达99.51亿元。科思股份的解禁规模次之，下周将有1.03亿股上市流通，解禁股主要为首发原股东限售股份，解禁市值达79.85亿元。科润智控、华康医疗、建邦科技等9股的解禁压力较小，解禁市值均不足千万元。解禁比例来看，国光连锁、科思股份、科拓生物的解禁比例均超50%。其中国光连锁的解禁股数量占总股本比例最大，达87.82%。此外，泽璟制药-U、晶澳科技、建邦科技、华康医疗、领湃科技的解禁比例较小，均不到0.1%。
2023-07-25 0

Processing rows:  50%|████▉     | 655/1314 [05:05<05:14,  2.10it/s]

...

Processing rows:  66%|██████▌   | 866/1314 [05:21<01:48,  4.13it/s]

Error occurred with prompt: 现在是2024-01-30 09:00:00+08:00
前五个交易日上证指数的价格如下2826.503,2746.892,2782.142,2823.826,2897.915

2024-01-25 21:06:41 新闻:【“中字头”股票大涨 央企密集发声回应“市值管理”】国资委将研究将市值管理纳入央企负责人业绩考核，“中字头”股票全线大涨。对此，中国中免、中海油服、中国化学等企业密集发生。（上证报）
2024-01-27 07:50:50 新闻:【国企改革概念成热点 基金提前结募迎风口】 据上证报，央企基金、国企基金的风口来了。在1月24日的发布会上，国务院国资委表示将研究将市值管理成效纳入中央企业负责人业绩考核，此番表态引起市场巨大反响，“中字头”个股强势拉升，让重仓这些标的的基金净值大涨，基金公司也快速调整策略以迎接风口。原定2月6日结束募集的汇添富中证国新央企股东回报基金，于1月31日即提前结束募集。
2024-01-24 14:24:27 新闻:【深圳国企改革午后冲高，特发信息此前涨停】深物业A、深深房A、建科院、沙河股份涨超4%，特发服务、国信证券、深高速、广聚能源上涨。
2024-01-11 13:39:25 新闻:【工程机械板块震荡走高 天桥起重涨停】 工程机械板块震荡走高，天桥起重涨停，拓山重工涨逾8%，山河智能涨逾7%，梅安森、天奇股份、安徽合力、山推股份等跟涨。
2024-01-09 07:25:56 新闻:【2023年54家A股公司披露分拆意向，制造业企业占比超八成】数据显示，2023年A股共有54家公司披露分拆意向，其中9单已成功分拆、13单终止。就母公司所属证监会行业来看，有47家制造业公司送子单飞，热情颇高，拟分拆业务多为战略新兴产业。就子公司拟上市板块来看，北交所吸引力渐显，有10家公司明确表示拟分拆至北交所，其中4家已成功上市。记者盘点终止案例，A股分拆降温的背后因素很多，不符合相关政策要求是最主要因素之一。“上市公司筹划分拆上市事项，主要还是看分拆上市的必要性，以及业务独立性、分拆出来的子公司是否符合相关板块定位。”上海国家会计学院金融系主任叶小杰表示。（上证报）
2024-01-21 08:13:33 新闻:【联博：境外长期资金对A股信心不变 今年全球债市进一步攀涨概率

Processing rows:  66%|██████▌   | 866/1314 [05:35<01:48,  4.13it/s]

Error occurred with prompt: 现在是2024-03-29 09:00:00+08:00
前五个交易日上证指数的价格如下3084.412,3070.548,3040.501,3026.922,3026.814

2024-03-11 05:11:33 新闻:【深耕主业叠加技术性举措 上市公司打出市值管理“组合拳”】业内人士普遍认为，通过完善治理、创新技术、开发新品、开拓市场等手段做好经营、提升业绩、回报投资者，是上市公司做好市值管理的根本。当然，面对市场波动，负责任的上市公司还应在合适时点推出重要股东增持、回购股份、加大分红力度等技术性市值管理举措，稳定股价，维护市值。
2024-03-11 05:12:12 新闻:【央国企料迎价值重估 券商看好投资机会】开年以来，央企市值管理成为市场最关注的焦点之一。券商人士认为，市值管理本质是国企改革的重要一环，改革的重点在于通过提升盈利能力、优化治理机制、强化管理层与资本市场的沟通等渠道提高企业核心竞争力与增强核心功能，这是做好央企市值管理的重要途径。机构认为，投资者要挖掘低估值央企投资机会，央企领军的15个重点产业领域方向值得关注。展望未来，随着市值管理的深化，央国企或迎来价值重估。（中证报）
2024-03-18 14:12:21 新闻:A股万达电影盘中涨停，成交额超13亿元。
2024-03-11 05:12:12 新闻:【央国企料迎价值重估 券商看好投资机会】开年以来，央企市值管理成为市场最关注的焦点之一。券商人士认为，市值管理本质是国企改革的重要一环，改革的重点在于通过提升盈利能力、优化治理机制、强化管理层与资本市场的沟通等渠道提高企业核心竞争力与增强核心功能，这是做好央企市值管理的重要途径。机构认为，投资者要挖掘低估值央企投资机会，央企领军的15个重点产业领域方向值得关注。展望未来，随着市值管理的深化，央国企或迎来价值重估。（中证报）

时间：2024-03-28 19:38:03
标题：中国中铁2023年度独立董事履职报告（钟瑞明）


 根据中国中铁2023年度独立董事履职报告，以下是关键信息的总结：

1. 钟瑞明是独立非执行董事，现任香港城市大学副校监，并担任多家上市公司的独立董事。

2. 2023年，钟瑞明出席股东大会1次，参加董事会会议13次，审议并表决通过78项议案。

3.

Processing rows: 100%|██████████| 1314/1314 [07:51<00:00,  2.79it/s]


In [37]:
with open(f'test_dataset/processed_{name}.json', 'w', encoding='utf-8') as f:
    json.dump(process_data, f, ensure_ascii=False, indent=4)